In [59]:
import numpy as np
import pandas as pd

# Evaluation libraries
from sklearn.metrics import accuracy_score, precision_score, f1_score ,recall_score


import tensorflow as tf
import keras
from keras.layers import Dense,Dropout
from keras.models import Sequential

from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 
from keras.utils import np_utils
from sklearn.model_selection import KFold 
from sklearn.preprocessing import MinMaxScaler

# keras-tuner optimization
import kerastuner as kt
from kerastuner.tuners import RandomSearch
from numpy import average
from keras import backend as K
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.callbacks import EarlyStopping

# Visualization Data
import matplotlib.pyplot as plt

### Reading  training set and testing set

In [60]:
train= pd.read_csv('train_datasetset1.csv',encoding='utf-8')
y_train = train.label                           
X_train= train.drop('label',axis = 1 )
train.label.value_counts()

B    285
M    170
Name: label, dtype: int64

In [61]:
test= pd.read_csv('unseen_dataset1.csv',encoding='utf-8')
y_test = test.label                           
X_test= test.drop('label',axis = 1 )
test.label.value_counts()

B    72
M    42
Name: label, dtype: int64

In [62]:
num_features=len(X_train.columns)

### Sacling dataset

In [63]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### OneHotEncoder

In [64]:
encoder = LabelEncoder()
encoded_y_train= encoder.fit_transform(y_train)
y_train_categorical = np_utils.to_categorical(encoded_y_train)
y_test = encoder.transform(y_test)
y_test_categorical=np_utils.to_categorical(y_test)

In [66]:
Epochs=50
Batch_size=10

### Evaluation methods

In [67]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

### KerasTuner 
#https://keras.io/keras_tuner/ 
KerasTuner is an easy-to-use, scalable hyperparameter optimization framework that solves the pain points of hyperparameter search. Easily configure your search space with a define-by-run syntax, then leverage one of the available search algorithms to find the best hyperparameter values for your models. KerasTuner comes with Bayesian Optimization, Hyperband, and Random Search algorithms built-in, and is also designed to be easy for researchers to extend in order to experiment with new search algorithms.
We 

In [68]:
def build_model(hp):
    num_units_min  =  50
    num_units_max  =  600
    num_units_step =  20

    dropout_min  =  .2
    dropout_max  =  0.9
    dropout_step =  0.1
    
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('unit1',  min_value=num_units_min,
                                                 max_value=num_units_max,
                                                 step=num_units_step),
                                                 input_dim=num_features, 
                                                kernel_initializer='he_uniform',
                           activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout_1',min_value=dropout_min,
                                      max_value=dropout_max,
                                      step=dropout_step) ) )
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(layers.Dense(units=hp.Int('unitfor_'+ str(i),
                                            min_value=num_units_min,
                                            max_value=num_units_max,
                                            step=num_units_step),
                                           kernel_initializer='he_uniform',activation='relu'          
                                         ))
        model.add(layers.Dropout(hp.Float('dropout_2'+ str(i),min_value=dropout_min,
                                      max_value=dropout_max,
                                      step=dropout_step) ) )
    
    model.add(layers.Dense(2, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[ 1e-4])),
       loss='binary_crossentropy', 
       metrics=['acc',f1_m,precision_m, recall_m])
    return model


In [69]:
# Determine project name and the number of iteration 
tuner = RandomSearch(
    build_model,
    objective='val_acc',
    max_trials=2,
    project_name='RNN')


INFO:tensorflow:Reloading Oracle from existing project .\RNN\oracle.json
INFO:tensorflow:Reloading Tuner from .\RNN\tuner0.json


In [70]:

early_stopping = EarlyStopping(monitor='val_acc', patience=30, verbose=1)
callback_list = [ early_stopping ]
h=tuner.search(X_train, y_train_categorical,
             epochs=Epochs,
             batch_size=Batch_size, 
             callbacks=callback_list, validation_data=(X_test,y_test_categorical) )
            
model = tuner.get_best_models(num_models=1)[0]


INFO:tensorflow:Oracle triggered exit


### get the best values of parameters

In [71]:

tuner.get_best_hyperparameters()[0].values

{'unit1': 70,
 'dropout_1': 0.4000000000000001,
 'num_layers': 2,
 'unitfor_0': 470,
 'dropout_20': 0.30000000000000004,
 'learning_rate': 0.0001,
 'unitfor_1': 50,
 'dropout_21': 0.2}

In [72]:
AccuracyTrain=[]
PrecisionTrain=[]
RecallTrain=[]
F1Train=[]
Roc_auc_Train=[]
AccuracyTest=[]
PrecisionTest=[]
RecallTest=[]
F1Test=[]

for i in range(0,4):    
    if i==0:
        r=70
    elif i== 1:
        r=80
    elif i== 2:
        r=90
    elif i== 3:
        r=100
    elif i== 4:
        r=140
    
    kf = KFold(n_splits=10,random_state=r,shuffle=True) 
    for train_index, test_index in kf.split(X_train):
        X_t, X_val = X_train[train_index], X_train[test_index] 
        y_t, y_val = y_train_categorical[train_index], y_train_categorical[test_index]
        h=model.fit(X_t, y_t, epochs=Epochs, batch_size=Batch_size,  verbose=1, shuffle=True, validation_data=(X_val,y_val))

        AccuracyTrain.append(round(100*np.mean(h.history['val_acc']),2))
        
        PrecisionTrain.append(round(100*np.mean(h.history['val_precision_m']),2))
       
        RecallTrain.append(round(100*np.mean(h.history['val_recall_m']),2))
        
        F1Train.append(round(100*np.mean(h.history['val_f1_m']),2))
        

Epoch 1/50
41/41 [==============================] - 1s 11ms/step - loss: 0.2112 - acc: 0.9071 - f1_m: 0.9116 - precision_m: 0.9153 - recall_m: 0.9095 - val_loss: 0.1419 - val_acc: 0.9348 - val_f1_m: 0.9229 - val_precision_m: 0.9073 - val_recall_m: 0.9400
Epoch 2/50
41/41 [==============================] - 0s 4ms/step - loss: 0.1839 - acc: 0.9340 - f1_m: 0.9295 - precision_m: 0.9396 - recall_m: 0.9217 - val_loss: 0.1401 - val_acc: 0.9130 - val_f1_m: 0.9064 - val_precision_m: 0.8781 - val_recall_m: 0.9400
Epoch 3/50
41/41 [==============================] - 0s 3ms/step - loss: 0.1696 - acc: 0.9438 - f1_m: 0.9436 - precision_m: 0.9463 - recall_m: 0.9415 - val_loss: 0.1414 - val_acc: 0.8913 - val_f1_m: 0.8971 - val_precision_m: 0.8885 - val_recall_m: 0.9067
Epoch 4/50
41/41 [==============================] - 0s 3ms/step - loss: 0.1717 - acc: 0.9438 - f1_m: 0.9451 - precision_m: 0.9450 - recall_m: 0.9461 - val_loss: 0.1444 - val_acc: 0.8913 - val_f1_m: 0.8867 - val_precision_m: 0.8867 - val_

41/41 [==============================] - 0s 3ms/step - loss: 0.1371 - acc: 0.9584 - f1_m: 0.9500 - precision_m: 0.9582 - recall_m: 0.9436 - val_loss: 0.0874 - val_acc: 0.9348 - val_f1_m: 0.9400 - val_precision_m: 0.9400 - val_recall_m: 0.9400
Epoch 34/50
41/41 [==============================] - 0s 3ms/step - loss: 0.1246 - acc: 0.9535 - f1_m: 0.9499 - precision_m: 0.9516 - recall_m: 0.9488 - val_loss: 0.0838 - val_acc: 0.9348 - val_f1_m: 0.9400 - val_precision_m: 0.9400 - val_recall_m: 0.9400
Epoch 35/50
41/41 [==============================] - 0s 3ms/step - loss: 0.1059 - acc: 0.9658 - f1_m: 0.9641 - precision_m: 0.9583 - recall_m: 0.9707 - val_loss: 0.0852 - val_acc: 0.9348 - val_f1_m: 0.9400 - val_precision_m: 0.9400 - val_recall_m: 0.9400
Epoch 36/50
41/41 [==============================] - 0s 3ms/step - loss: 0.1247 - acc: 0.9487 - f1_m: 0.9524 - precision_m: 0.9538 - recall_m: 0.9512 - val_loss: 0.0886 - val_acc: 0.9348 - val_f1_m: 0.9267 - val_precision_m: 0.9267 - val_recall_m:

41/41 [==============================] - 0s 3ms/step - loss: 0.1080 - acc: 0.9633 - f1_m: 0.9645 - precision_m: 0.9638 - recall_m: 0.9656 - val_loss: 0.0257 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 16/50
41/41 [==============================] - 0s 3ms/step - loss: 0.0968 - acc: 0.9584 - f1_m: 0.9562 - precision_m: 0.9521 - recall_m: 0.9607 - val_loss: 0.0215 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 17/50
41/41 [==============================] - 0s 3ms/step - loss: 0.0878 - acc: 0.9707 - f1_m: 0.9697 - precision_m: 0.9690 - recall_m: 0.9707 - val_loss: 0.0183 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 18/50
41/41 [==============================] - 0s 4ms/step - loss: 0.1155 - acc: 0.9584 - f1_m: 0.9594 - precision_m: 0.9584 - recall_m: 0.9607 - val_loss: 0.0181 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m:

41/41 [==============================] - 0s 3ms/step - loss: 0.1412 - acc: 0.9462 - f1_m: 0.9425 - precision_m: 0.9416 - recall_m: 0.9436 - val_loss: 0.0159 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 48/50
41/41 [==============================] - 0s 3ms/step - loss: 0.0961 - acc: 0.9707 - f1_m: 0.9693 - precision_m: 0.9707 - recall_m: 0.9683 - val_loss: 0.0176 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 49/50
41/41 [==============================] - 0s 4ms/step - loss: 0.0747 - acc: 0.9780 - f1_m: 0.9765 - precision_m: 0.9780 - recall_m: 0.9753 - val_loss: 0.0172 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 50/50
41/41 [==============================] - 0s 3ms/step - loss: 0.0948 - acc: 0.9609 - f1_m: 0.9634 - precision_m: 0.9638 - recall_m: 0.9634 - val_loss: 0.0210 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m:

41/41 [==============================] - 0s 3ms/step - loss: 0.0689 - acc: 0.9756 - f1_m: 0.9716 - precision_m: 0.9753 - recall_m: 0.9683 - val_loss: 0.0260 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 30/50
41/41 [==============================] - 0s 3ms/step - loss: 0.0837 - acc: 0.9707 - f1_m: 0.9672 - precision_m: 0.9665 - recall_m: 0.9683 - val_loss: 0.0285 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 31/50
41/41 [==============================] - 0s 3ms/step - loss: 0.0800 - acc: 0.9780 - f1_m: 0.9780 - precision_m: 0.9780 - recall_m: 0.9780 - val_loss: 0.0252 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 32/50
41/41 [==============================] - 0s 3ms/step - loss: 0.0930 - acc: 0.9658 - f1_m: 0.9635 - precision_m: 0.9639 - recall_m: 0.9634 - val_loss: 0.0259 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m:

...

Epoch 5/50
41/41 [==============================] - 0s 4ms/step - loss: 0.0087 - acc: 0.9976 - f1_m: 0.9976 - precision_m: 0.9976 - recall_m: 0.9976 - val_loss: 1.8500e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 6/50
41/41 [==============================] - 0s 3ms/step - loss: 6.8092e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 2.2398e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 7/50
41/41 [==============================] - 0s 4ms/step - loss: 4.4053e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 2.9026e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 8/50
41/41 [==============================] - 0s 4ms/step - loss: 0.0016 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 1.0574e-04 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_pre

41/41 [==============================] - 0s 3ms/step - loss: 0.0105 - acc: 0.9976 - f1_m: 0.9976 - precision_m: 0.9976 - recall_m: 0.9976 - val_loss: 1.2438e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 37/50
41/41 [==============================] - 0s 3ms/step - loss: 0.0024 - acc: 0.9976 - f1_m: 0.9976 - precision_m: 0.9976 - recall_m: 0.9976 - val_loss: 2.3525e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 38/50
41/41 [==============================] - 0s 4ms/step - loss: 0.0026 - acc: 0.9976 - f1_m: 0.9976 - precision_m: 0.9976 - recall_m: 0.9976 - val_loss: 7.1014e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 39/50
41/41 [==============================] - 0s 3ms/step - loss: 5.3204e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 6.3672e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.

Epoch 18/50
41/41 [==============================] - 0s 3ms/step - loss: 0.0015 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 0.0070 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 19/50
41/41 [==============================] - 0s 4ms/step - loss: 0.0011 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 0.0048 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 20/50
41/41 [==============================] - 0s 3ms/step - loss: 8.2039e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 0.0039 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 21/50
41/41 [==============================] - 0s 4ms/step - loss: 0.0011 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 0.0039 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000

Epoch 50/50
41/41 [==============================] - 0s 4ms/step - loss: 0.0060 - acc: 0.9976 - f1_m: 0.9976 - precision_m: 0.9976 - recall_m: 0.9976 - val_loss: 0.0054 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 1/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0020 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 5.0772e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 2/50
41/41 [==============================] - 0s 5ms/step - loss: 0.0104 - acc: 0.9976 - f1_m: 0.9976 - precision_m: 0.9976 - recall_m: 0.9976 - val_loss: 5.4306e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 3/50
41/41 [==============================] - 0s 4ms/step - loss: 0.0089 - acc: 0.9951 - f1_m: 0.9951 - precision_m: 0.9951 - recall_m: 0.9951 - val_loss: 1.3992e-04 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1

41/41 [==============================] - 0s 4ms/step - loss: 0.0055 - acc: 0.9951 - f1_m: 0.9951 - precision_m: 0.9951 - recall_m: 0.9951 - val_loss: 1.6960e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 32/50
41/41 [==============================] - 0s 4ms/step - loss: 0.0045 - acc: 0.9976 - f1_m: 0.9976 - precision_m: 0.9976 - recall_m: 0.9976 - val_loss: 2.8057e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 33/50
41/41 [==============================] - 0s 4ms/step - loss: 0.0020 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 2.7551e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 34/50
41/41 [==============================] - 0s 4ms/step - loss: 0.0036 - acc: 0.9976 - f1_m: 0.9976 - precision_m: 0.9976 - recall_m: 0.9976 - val_loss: 3.7000e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000

In [75]:
for i in range(0,4): 
    model.fit(X_train, y_train_categorical, epochs=Epochs, batch_size=Batch_size,  verbose=1, shuffle=True)
    y_p = model.predict(X_test)
    y_pred= np.argmax(y_p, axis=1)
    

    Accurcy_Test= accuracy_score(y_test,y_pred)
    Precision_Test=precision_score(y_test, y_pred,average='weighted')

    Recall_Test=recall_score(y_test, y_pred, average='weighted')
    F1_Test=f1_score(y_test, y_pred, average='weighted') 

    AccuracyTest.append(round(100*Accurcy_Test, 2))
    PrecisionTest.append(round(100*Precision_Test, 2))
    RecallTest.append(round(100*Recall_Test, 2))
    F1Test.append(round(100*F1_Test, 2))
    

Epoch 1/50
46/46 [==============================] - 0s 2ms/step - loss: 6.2990e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 2/50
46/46 [==============================] - 0s 2ms/step - loss: 0.0020 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 3/50
46/46 [==============================] - 0s 3ms/step - loss: 2.8008e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 4/50
46/46 [==============================] - 0s 3ms/step - loss: 0.0020 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 5/50
46/46 [==============================] - 0s 2ms/step - loss: 0.0076 - acc: 0.9956 - f1_m: 0.9957 - precision_m: 0.9957 - recall_m: 0.9957
Epoch 6/50
46/46 [==============================] - 0s 3ms/step - loss: 3.4308e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 7/50
46/46 [==============================] - 0s 3ms/step - loss: 5.0820e-04 - acc: 1.00

Epoch 5/50
46/46 [==============================] - 0s 3ms/step - loss: 2.8038e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 6/50
46/46 [==============================] - 0s 3ms/step - loss: 0.0013 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 7/50
46/46 [==============================] - 0s 2ms/step - loss: 8.0999e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 8/50
46/46 [==============================] - 0s 2ms/step - loss: 0.0019 - acc: 1.0000 - f1_m: 0.9989 - precision_m: 1.0000 - recall_m: 0.9978
Epoch 9/50
46/46 [==============================] - 0s 3ms/step - loss: 0.0021 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 10/50
46/46 [==============================] - 0s 3ms/step - loss: 0.0079 - acc: 0.9978 - f1_m: 0.9978 - precision_m: 0.9978 - recall_m: 0.9978
Epoch 11/50
46/46 [==============================] - 0s 3ms/step - loss: 3.5285e-04 - acc: 1.0000

46/46 [==============================] - 0s 2ms/step - loss: 0.0014 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 9/50
46/46 [==============================] - 0s 3ms/step - loss: 0.0013 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 10/50
46/46 [==============================] - 0s 2ms/step - loss: 3.6447e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 11/50
46/46 [==============================] - 0s 3ms/step - loss: 0.0030 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 12/50
46/46 [==============================] - 0s 2ms/step - loss: 0.0024 - acc: 0.9978 - f1_m: 0.9978 - precision_m: 0.9978 - recall_m: 0.9978
Epoch 13/50
46/46 [==============================] - 0s 2ms/step - loss: 0.0028 - acc: 0.9978 - f1_m: 0.9978 - precision_m: 0.9978 - recall_m: 0.9978
Epoch 14/50
46/46 [==============================] - 0s 3ms/step - loss: 0.0243 - acc: 0.9912 - f1_m: 0.9913 

46/46 [==============================] - 0s 3ms/step - loss: 6.0642e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 13/50
46/46 [==============================] - 0s 3ms/step - loss: 0.0054 - acc: 0.9978 - f1_m: 0.9978 - precision_m: 0.9980 - recall_m: 0.9978
Epoch 14/50
46/46 [==============================] - 0s 2ms/step - loss: 0.0019 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 15/50
46/46 [==============================] - 0s 3ms/step - loss: 4.8428e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 16/50
46/46 [==============================] - 0s 2ms/step - loss: 0.0036 - acc: 0.9978 - f1_m: 0.9978 - precision_m: 0.9978 - recall_m: 0.9978
Epoch 17/50
46/46 [==============================] - 0s 2ms/step - loss: 6.8625e-04 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 18/50
46/46 [==============================] - 0s 3ms/step - loss: 0.0098 - acc: 0.9978 - f1_m

In [76]:
finaltrain = pd.DataFrame([])

finaltrain = finaltrain.append({'AccuracyTrain' : round(np.mean(AccuracyTrain),2),
                                'PrecisionTrain':round(np.mean(PrecisionTrain),2),
                                'RecallTrain':round(np.mean(RecallTrain),2),  
                                'F1Train':round(np.mean(F1Train),2)} , ignore_index=True)

finaltrain.reindex(['AccuracyTrain','PrecisionTrain','RecallTrain','F1Train'], axis=1)

C:\Users\hager\AppData\Local\Temp/ipykernel_20044/3956621564.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finaltrain = finaltrain.append({'AccuracyTrain' : round(np.mean(AccuracyTrain),2),


,AccuracyTrain,PrecisionTrain,RecallTrain,F1Train
0,99.45,99.43,99.44,99.43


In [77]:
ReultofTest=pd.DataFrame([])
ReultofTest=ReultofTest.append({'AccuracyTest' : round(np.mean(AccuracyTest),2),'PrecisionTest':round(np.mean(PrecisionTest),2),
             'RecallTest' : round(np.mean(RecallTest),2),'F1Test':round(np.mean(F1Test),2)}, ignore_index=True)

ReultofTest.reindex(['AccuracyTest','PrecisionTest','RecallTest','F1Test'], axis=1)

C:\Users\hager\AppData\Local\Temp/ipykernel_20044/1801123747.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ReultofTest=ReultofTest.append({'AccuracyTest' : round(np.mean(AccuracyTest),2),'PrecisionTest':round(np.mean(PrecisionTest),2),


,AccuracyTest,PrecisionTest,RecallTest,F1Test
0,94.08,94.42,94.08,93.98
